In [ ]:
import numpy as np
import pandas as pd
import glob
import os
#for dirname, _, filenames in os.walk('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.optimizers import Adam, SGD
import pandas as pd

In [ ]:
Mild = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/MildDemented/*.*')
Moderate = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/ModerateDemented/*.*')
NonDemented = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/NonDemented/*.*')
VeryMild = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/VeryMildDemented/*.*')


data = []
labels = []

for i in NonDemented:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append(0)

for i in VeryMild:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append(1)

for i in Mild:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append(2)

for i in Moderate:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224))
    image=np.array(image)
    data.append(image)
    labels.append(3)

In [ ]:
import glob
Mild_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/MildDemented/*.*')
Moderate_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/ModerateDemented/*.*')
NonDemented_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/NonDemented/*.*')
VeryMild_test = glob.glob('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test/VeryMildDemented/*.*')


data_test = []
labels_test  = []

for i in NonDemented_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),grayscale=False)
    image=np.array(image)
    data_test.append(image)
    labels_test.append(0)

for i in VeryMild_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),grayscale=False)
    image=np.array(image)
    data_test.append(image)
    labels_test.append(1)

for i in Mild_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),grayscale=False)
    image=np.array(image)
    data_test.append(image)
    labels_test.append(2)

for i in Moderate_test:   
    image=tf.keras.preprocessing.image.load_img(i, target_size= (224,224),grayscale=False)
    image=np.array(image)
    data_test.append(image)
    labels_test.append(3)

In [ ]:
data = np.array(data)
labels = np.array(labels)

data_test = np.array(data_test)
labels_test = np.array(labels_test)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255,
                            rotation_range=45,
                            width_shift_range=.15,
                            height_shift_range=.15,
                            horizontal_flip=False)

train_data_gen = datagen.flow(
    x=data,
    y=labels,
    batch_size=64,
)

In [ ]:
test_data_gen = datagen.flow(
    x=data_test,
    y=labels_test
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Layer

class SelfAttention(Layer):
    def __init__(self):
        super(SelfAttention, self).__init__()

    def build(self, input_shape):
        self.W_q = self.add_weight(shape=(input_shape[-1], input_shape[-1]),
                                  initializer='uniform',
                                  trainable=True)
        self.W_k = self.add_weight(shape=(input_shape[-1], input_shape[-1]),
                                  initializer='uniform',
                                  trainable=True)
        super(SelfAttention, self).build(input_shape)

    def call(self, x):
        Q = tf.matmul(x, self.W_q)
        K = tf.matmul(x, self.W_k)
        V = x
        attention_weights = tf.nn.softmax(tf.matmul(Q, K, transpose_b=True), axis=-1)
        output = tf.matmul(attention_weights, V)
        return output

# Create the model with the Self-Attention Layer
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    SelfAttention(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_data_gen, batch_size=32, epochs=10, validation_data=test_data_gen)